### Initialization of Key Tables, Add to SQLite DB

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os

from nba_api.stats.static import players
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

from utils import * 

In [2]:
db_path = './sqlite/nba_db'

#### NBA Player Info

In [2]:
# Define SQL schema for player_bio

create_table = """

    CREATE TABLE player_bio (

        id INT PRIMARY KEY, 
        full_name VARCHAR(255) NOT NULL, 
        first_name VARCHAR(255) NOT NULL, 
        last_name VARCHAR(255) NOT NULL, 
        is_active INT NOT NULL

    );

    """


In [3]:
query_table = """
    SELECT * FROM player_bio
"""

In [4]:
results = query_db(db_path, query_table)

In [5]:
print(results)

[]


In [14]:
# Get NBA player bio

player_bio_data = players.get_players()

#player_bio = pd.DataFrame(player_bio)
#nba_players = nba_players.loc[nba_players.is_active == True]
#nba_players = nba_players[['id', 'full_name']]

sample_data = player_bio_data[:2]

print(sample_data)

[{'id': 76001, 'full_name': 'Alaa Abdelnaby', 'first_name': 'Alaa', 'last_name': 'Abdelnaby', 'is_active': False}, {'id': 76002, 'full_name': 'Zaid Abdul-Aziz', 'first_name': 'Zaid', 'last_name': 'Abdul-Aziz', 'is_active': False}]


In [15]:
sample_data.keys()

AttributeError: 'list' object has no attribute 'keys'

In [6]:
create_table_from_df(db_path, 'player_bio', player_bio)

Table 'player_bio' created from DataFrame successfully.


In [15]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('select * from player_bio').fetchall()[:5]
#print(cursor[:1])

#conn.close()

[(76001, 'Alaa Abdelnaby', 'Alaa', 'Abdelnaby', 0),
 (76002, 'Zaid Abdul-Aziz', 'Zaid', 'Abdul-Aziz', 0),
 (76003, 'Kareem Abdul-Jabbar', 'Kareem', 'Abdul-Jabbar', 0),
 (51, 'Mahmoud Abdul-Rauf', 'Mahmoud', 'Abdul-Rauf', 0),
 (1505, 'Tariq Abdul-Wahad', 'Tariq', 'Abdul-Wahad', 0)]

In [28]:
# Mapping Table for IDs -> Teams

nba_teams = teams.get_teams()
nba_teams = pd.DataFrame(nba_teams)

nba_teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


#### Game Agg Stats

In [29]:
# Get All Games

min_date = '2000-01-01'

nba_team_ids = list(nba_teams.id)
all_games = pd.DataFrame()

for id in nba_team_ids:
    
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = id)
    tmp = gamefinder.get_data_frames()[0]
    tmp = tmp.loc[tmp.GAME_DATE >= min_date]

    all_games = pd.concat([all_games, tmp])


# Manual adjustment for NBA Cup for subsequent combine_team_games() to work properly
all_games.loc[(all_games.GAME_ID == '0062400001') & (all_games.TEAM_ABBREVIATION == 'MIL'), 'MATCHUP'] = 'MIL vs. OKC'

#

all_games['TEAM_SIDE'] = np.where(all_games.MATCHUP.str.contains(' @ '), 'awayTeam', 'homeTeam')

# Create season_short

all_games['SEASON_SHORT'] = all_games.SEASON_ID.str[-4:].astype('int')


# Get game types, requires some manual mapping

game_type_dict = {
            '1': 'preseason', 
            '2': 'regular', 
            '4': 'playoffs', 
            '5': 'play-in', 
            '6': 'nba_cup'}

all_games['GAME_TYPE'] = all_games.SEASON_ID.str[:1].map(game_type_dict)

reg_start_date = {
    2000: '2000-10-31',
    2001: '2001-10-30',
    2002: '2002-10-29',
    2003: '2003-10-28',
    2004: '2004-10-26',
    2005: '2005-11-01',
    2006: '2006-11-01',
    2007: '2007-10-30',
    2008: '2008-10-28',
    2009: '2009-10-27',
    2010: '2010-10-26',
    2011: '2011-12-25',
    2012: '2012-10-30',
    2013: '2013-10-29',
    2014: '2014-10-28',
    2015: '2015-10-27',
    2016: '2016-10-25',
    2017: '2017-10-17',
    2018: '2018-10-16',
    2019: '2019-10-22',
    2020: '2020-12-22',
    2021: '2021-10-19',
    2022: '2022-10-18',
    2023: '2023-10-24',
    2024: '2024-10-22'}

for _, val in enumerate(reg_start_date):

    all_games.loc[(all_games.GAME_TYPE == 'regular') & (all_games.SEASON_SHORT == val) & (all_games.GAME_DATE < reg_start_date[val]), 'GAME_TYPE'] = 'preseason'


all_games = all_games.drop(['TEAM_NAME'], axis = 1).rename(columns = {'TEAM_ABBREVIATION': 'TEAM'})

# Get opponent
all_games['OPPONENT'] = all_games.MATCHUP.str[-3:]


all_games.loc[all_games.GAME_ID == '0022400477'].head()

,SEASON_ID,TEAM_ID,TEAM,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,...,AST,STL,BLK,TOV,PF,PLUS_MINUS,TEAM_SIDE,SEASON_SHORT,GAME_TYPE,OPPONENT
1,22024,1610612737,ATL,0022400477,2025-01-03,ATL @ LAL,L,240,102,41,...,22,10.0,3,8,22,-17.0,awayTeam,2024,regular,LAL
1,22024,1610612747,LAL,0022400477,2025-01-03,LAL vs. ATL,W,240,119,43,...,28,4.0,6,18,21,17.0,homeTeam,2024,regular,ATL


In [30]:
all_games.to_parquet('./data/all_games.parquet')

In [31]:
# Get single line game stats via combine_team_games()
# _H represents stats relating to the home team, _A reptesents away team

all_games_comb = combine_team_games(all_games)

all_games_comb = all_games_comb.drop(['GAME_TYPE_A', 'SEASON_SHORT_A', 'MATCHUP_A', 'TEAM_SIDE_A', 'TEAM_SIDE_H', 'OPPONENT_A', 'OPPONENT_H'], axis = 1)
all_games_comb = all_games_comb.rename(columns={'GAME_TYPE_H': 'GAME_TYPE', 'SEASON_SHORT_H': 'SEASON_SHORT', 'MATCHUP_H': 'MATCHUP'})

all_games_comb.loc[all_games_comb.GAME_ID == '0022400477'].head()


,SEASON_ID,TEAM_ID_H,TEAM_H,GAME_ID,GAME_DATE,MATCHUP,WL_H,MIN_H,PTS_H,FGM_H,...,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A
6,22024,1610612747,LAL,0022400477,2025-01-03,LAL vs. ATL,W,240,119,43,...,0.778,13.0,26.0,39.0,22,10.0,3,8,22,-17.0


In [32]:
all_games_comb.to_parquet('./data/all_games_comb.parquet')

#### Validation

In [33]:
# Check all_games df has 82 games per team per season

team_list = list(nba_teams.abbreviation)
season = 2018

for team in team_list:
    tmp = all_games.loc[(all_games.TEAM == team) & (all_games.GAME_TYPE == 'regular') & (all_games.SEASON_SHORT == season)]
    print('{}: {}'.format(team, tmp.shape[0]))

ATL: 82
BOS: 82
CLE: 82
NOP: 82
CHI: 82
DAL: 82
DEN: 82
GSW: 82
HOU: 82
LAC: 82
LAL: 82
MIA: 82
MIL: 82
MIN: 82
BKN: 82
NYK: 82
ORL: 82
IND: 82
PHI: 82
PHX: 82
POR: 82
SAC: 82
SAS: 82
OKC: 82
TOR: 82
UTA: 82
MEM: 82
WAS: 82
DET: 82
CHA: 82


In [34]:
# Check all_games_comb df has 82 games per team per season

team_list = list(nba_teams.abbreviation)
season = 2014

for team in team_list:
    tmp = all_games_comb.loc[((all_games_comb.TEAM_A == team) | (all_games_comb.TEAM_H == team)) & (all_games_comb.GAME_TYPE == 'regular') & (all_games_comb.SEASON_SHORT == season)]
    print('{}: {}'.format(team, tmp.shape[0]))

ATL: 82
BOS: 82
CLE: 82
NOP: 82
CHI: 82
DAL: 82
DEN: 82
GSW: 82
HOU: 82
LAC: 82
LAL: 82
MIA: 82
MIL: 82
MIN: 82
BKN: 82
NYK: 82
ORL: 82
IND: 82
PHI: 82
PHX: 82
POR: 82
SAC: 82
SAS: 82
OKC: 82
TOR: 82
UTA: 82
MEM: 82
WAS: 82
DET: 82
CHA: 82


#### Player Stats Per Game

In [40]:

all_games.loc[all_games.GAME_ID == '1522400064'].head()

,SEASON_ID,TEAM_ID,TEAM,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,...,AST,STL,BLK,TOV,PF,PLUS_MINUS,TEAM_SIDE,SEASON_SHORT,GAME_TYPE,OPPONENT
40,22024,1610612737,ATL,1522400064,2024-07-20,ATL vs. NYK,L,201,82,27,...,19,7.0,3,17,25,-6.6,homeTeam,2024,preseason,NYK
41,22024,1610612752,NYK,1522400064,2024-07-20,NYK @ ATL,W,201,90,31,...,15,8.0,5,18,16,7.0,awayTeam,2024,preseason,ATL


In [45]:
target_stats = ['assists', 'blocks', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'fieldGoalsPercentage', 'freeThrowsAttempted',
       'freeThrowsMade', 'freeThrowsPercentage',
       'minutesCalculated', 'plusMinusPoints', 'points',
       'pointsFastBreak', 'pointsInThePaint',
       'reboundsDefensive', 'reboundsOffensive', 'reboundsTotal', 'steals',
       'threePointersAttempted', 'threePointersMade',
       'threePointersPercentage', 'turnovers', 'twoPointersAttempted',
       'twoPointersMade', 'twoPointersPercentage']

target_games = list(all_games.GAME_ID.drop_duplicates())
#target_games = ['1522400064']
all_games_small = all_games[['TEAM', 'OPPONENT', 'GAME_ID', 'SEASON_SHORT', 'GAME_TYPE', 'TEAM_SIDE']]
player_game_stats = pd.DataFrame()
exceptions = []

for game_id in target_games:

       try:
              box = boxscore.BoxScore(game_id) 
       except:
              exceptions.append(game_id)
              continue

       for side in ['homeTeam', 'awayTeam']:
              print(game_id)
              
              tmp = pd.DataFrame(box.get_dict()['game'][side]['players'])[['status', 'personId', 'jerseyNum', 'position', 'starter', 'played', 'statistics', 'name']]
              tmp['GAME_ID'] = game_id
              
              # Expand statistics column
              statistics = pd.json_normalize(tmp['statistics'])[target_stats]
              tmp = tmp.drop(columns=['statistics'])
              tmp = pd.concat([tmp, statistics], axis=1)

              # Merge onto all_games
              tmp = tmp.merge(all_games_small.loc[all_games_small.TEAM_SIDE == side], on='GAME_ID')

              player_game_stats = pd.concat([player_game_stats, tmp])


player_game_stats.head()

0022400486
0022400486
0022400477
0022400477
0022400461
0022400461
0022400438
0022400438
0022400427
0022400427
0022400413
0022400413
0022400395
0022400395
0022400378
0022400378
0022400370
0022400370
0022401229
0022401229
0022401202
0022401202
0022400350
0022400350
0022400334
0022400334
0022400323
0022400323
0022400315
0022400315
0022400300
0022400300
0022400041
0022400041
0022400287
0022400287
0022400280
0022400280
0022400030
0022400030
0022400258
0022400258
0022400250
0022400250
0022400239
0022400239
0022400012
0022400012
0022400001
0022400001
0022400198
0022400198
0022400185
0022400185
0022400171
0022400171
0022400157
0022400157
0022400152
0022400152
0022400135
0022400135
0022400121
0022400121
0022400103
0022400103
0022400100
0022400100
0022400079
0022400079
0022400064
0022400064
0012400064
0012400064
0012400025
0012400025
0012400046
0012400046
0012400018
0012400018
0052300111
0052300111
0022301188
0022301188
0022301178
0022301178
0022301159
0022301159
0022301147
0022301147
0022301130

,status,personId,jerseyNum,position,starter,played,name,GAME_ID,assists,blocks,...,threePointersPercentage,turnovers,twoPointersAttempted,twoPointersMade,twoPointersPercentage,TEAM,OPPONENT,SEASON_SHORT,GAME_TYPE,TEAM_SIDE
0,ACTIVE,1626181,24,SF,1,1,Norman Powell,0022400486,1,0,...,0.4,2,6,4,0.666667,LAC,ATL,2024,regular,homeTeam
1,ACTIVE,202695,2,PF,1,1,Kawhi Leonard,0022400486,1,0,...,0.6,2,6,1,0.166667,LAC,ATL,2024,regular,homeTeam
2,ACTIVE,1627826,40,C,1,1,Ivica Zubac,0022400486,4,1,...,0.0,1,11,9,0.818182,LAC,ATL,2024,regular,homeTeam
3,ACTIVE,1627739,8,SG,1,1,Kris Dunn,0022400486,0,1,...,0.0,3,2,1,0.500000,LAC,ATL,2024,regular,homeTeam
4,ACTIVE,201935,1,PG,1,1,James Harden,0022400486,15,0,...,0.2,4,4,2,0.500000,LAC,ATL,2024,regular,homeTeam


In [47]:
player_game_stats.to_parquet('./data/player_game_stats.parquet')

In [43]:

box = boxscore.BoxScore('1522400064') 

#tmp = pd.DataFrame(box.get_dict()['game']['awayTeam']['players'])
#tmp['GAME_ID'] = game_id


#tmp.head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [19]:
box = boxscore.BoxScore(game_id) 
              
tmp = pd.DataFrame(box.get_dict()['game'][side]['players'])[['status', 'personId', 'jerseyNum', 'position', 'starter', 'played', 'statistics', 'name']]
tmp['GAME_ID'] = game_id

Index(['status', 'order', 'personId', 'jerseyNum', 'position', 'starter',
       'oncourt', 'played', 'statistics', 'name', 'nameI', 'firstName',
       'familyName', 'notPlayingReason', 'GAME_ID'],
      dtype='object')

In [39]:

# Step 3: Concatenate the parsed stats back into the original DataFrame

# Step 4: Clean up the column names (optional)
# This step ensures that the column names are more readable and consistent
df.columns = [col.replace('statistics.', '') for col in df.columns]